# OpenAI agents playground notebook

Test and play with the following in this notebook:

* OpenAI API calling
* Creating agent tools
* Controling agent flow

## Setup

In [1]:
# imports
from openai import OpenAI
import json

In [ ]:
#API key
OPENAI_API_KEY = 

## OpenAI API Basics

### Our first LLM query

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
message = "What is the capital of Namibia"

messages = [
    {"role": "user", "content": message},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

In [ ]:
response

In [ ]:
## Inspecting the response
print(json.dumps(response.model_dump(), indent=2))

In [ ]:
print(response.choices[0].message.content)

 ### System prompt

In [ ]:
SYSTEM_PROMPT = "Your are a helpful assistant knowledgeable about Namibia, please answer tourist queries in the most helpful manner."

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is the capital of Namibia"},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(response.choices[0].message.content)

### Chat history

In [ ]:
messages = [
    {"role": "user", "content": "Give me a random number"},
]

response_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"First answer: {response_1.choices[0].message.content}")

messages = [
    {"role": "user", "content": "Now add 10 to that number"},
]

response_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"Second answer: {response_2.choices[0].message.content}")

In [ ]:
messages = [
    {"role": "user", "content": "Give me a random number"},
]

response_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

messages.extend([
    {"role": "assistant", "content": response_1.choices[0].message.content},
    {"role": "user", "content": "Now add 10 to that number"},
])

response_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"First response: {response_1.choices[0].message.content}")
print(f"Second response: {response_2.choices[0].message.content}")

### Hyperparameters

### max_completion_tokens

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is the capital of Namibia"},
    ],
    max_completion_tokens=1
)

print(response.choices[0].message.content)

### timeout

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is the capital of Namibia?"}],
    timeout=0.1,
)

In [ ]:
# handling the timeout
from openai._exceptions import APITimeoutError

try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "What is the capital of Namibia?"}],
        timeout=2  # Enforce a 2-second timeout
    )
    print(response)
except APITimeoutError:
    print("The request timed out. Try reducing the response complexity or increasing the timeout.")

### temperature

In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 0.0,
    )
    print(response.choices[0].message.content)

In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 1.0,
    )
    print(response.choices[0].message.content)

In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 2.0,
    )
    print(response.choices[0].message.content)

## Agents

### Tools

In [ ]:
def calculator(expression):
    """
    Evaluate a mathematical expression containing only basic arithmetic operators. 

    Args:
        expression (str): The expression to evaluate.
    
    Returns:
        float: The result of the expression.
    
    """
    return eval(expression, {"__builtins__": None})

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": calculator.__name__,
        "description": calculator.__doc__,
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {"type": "string"},
            },
            "required": ["expression"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What is 72953 times 21094"
    }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

response.choices[0]

In [ ]:
response.choices[0].finish_reason

In [ ]:
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:
args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
result = calculator(**args)
print(result)

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
messages.append(
        {
        "role": "tool",
        "tool_call_id": response.choices[0].message.tool_calls[0].id,
        "content": str(result)
    }
)

In [ ]:
messages

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

response.choices[0].message.content

Let's try without tools:

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What is 72953*21094"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

response.choices[0].message.content

### Multiple tool calls:

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What is 72953*21094? What is 2314-1955"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

response.choices[0].message.tool_calls

### Weather tool:

In [ ]:
import requests

In [ ]:
def get_weather(latitude, longitude):
    """
    Get the current temperature and wind speed at a given location.

    Args:
        latitude (float): The latitude of the location.
        longitude (float): The longitude of the location.
        
    Returns:
        dict: A dictionary containing the temperature [C] and wind speed [km/h].
        
    """
    
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m")
    data = response.json()
    return data["current"]

In [ ]:
tools.append({
    "type": "function",
    "function": {
        "name": 
        "description": 
        "parameters": {
            "type": "object",
            "properties": {
                
                
            },
            "required": [],
            "additionalProperties": False
        },
        "strict": True
    }
})

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What is the weather in Windhoek?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
print(response.choices[0].finish_reason)
print(response.choices[0].message.tool_calls[0].function.name)
args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
print(args)

In [ ]:
result = get_weather(**args)

In [ ]:
response.choices[0]

In [ ]:
messages.append(response.choices[0].message)
messages.append({
    "role": "tool",
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
    "content": str(result)
})

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
print(response.choices[0].message.content)

## Build the agent

Lets put all we learned together and finish the agent!

In [ ]:
from openai import ChatCompletion

class Agent():
    def __init__(
        self,
        system_prompt: str,
        model: str,
        tools: list
    ):
        pass

    def _parse_response(self, response: ChatCompletion) -> dict:
        # check if we have a tool call
        if response.choices[0].finish_reason == 'tool_calls':
            tool_call =  response.choices[0].message.tool_calls[0]
            args = json.loads(tool_call.function.arguments)
            # check what function to call
            if tool_call.function.name == 'calculator':
                result = calculator(**args)
                next_step = {
                    'target': 'llm',
                    'message': {
                        'id': tool_call.id,
                        'content': str(result)
                    }
                }

            return next_step

    def flush(self):
        pass

    def run(self, query: str) -> str:
        # add the new user query to our messages
        self._messages.append({
            "role": "user",
            "content": query
        })
        for _ in range(10):
            response = client.chat.completions.create(
                model=self._model,
                messages=self._messages,
                tools=tools,
            )

            self._messages.append(response.choices[0].message)
            next_step = self.parse_output(response)

In [ ]:
SYSTEM_PROMPT = None

agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    model="gpt-4o-mini",
    tools=tools
)

In [ ]:
agent.flush()
agent.run("What is 2142 times 21344?")